# Trim CGM Data Script

Utilizes each patient's summary, glucose, ECG, and breathing tables. Not meant to use on patient Day 1 data due to insertion trauma and elevated noise levels.

In [3]:
import pandas as pd
import numpy as np
import math
from datetime import datetime, timedelta

In [20]:
def trimCGM(files:list, day:int):
    """Takes in a list of paths to csv files. Writes the trimmed data to a folder, which we will import into MATLAB for making and L-tuple SGD.
    csv files list should be in the order of Summary, ECG, Breathing, and Glucose tables.
    Returns a list of 2 dataframes, qualities which is a joined Summary, ECG, and Breathing table in the first index and the correspoinding Glucose
    table in the second index."""

    # read in the 4 tables for Patient X on Day Y
    summary = pd.read_csv(files[0])
    ecg = pd.read_csv(files[1])
    breathing = pd.read_csv(files[2])
    glucose = pd.read_csv(files[3])



    # ----- CLEANIGN UP GLUCOSE TABLE -----
    # filter out all manual readings
    glucose = glucose[glucose.type != 'manual']

    # create a string for the day number you want to filter glucose table for
    date_str = f'2014-10-0{day}'

    # filter glucose table for indicated day only
    glucose = glucose[glucose.date == date_str]

    # combine the strings from date and time coulmns and turm them into datetime objects
    glucose['Time'] = glucose.date + " " + glucose.time
    glucose = glucose.loc[:, ['Time', 'glucose', 'type']]

    # convert Time column into datetime objects
    glucose['Time'] = pd.to_datetime(glucose['Time'])

    

    # ----- CLEANING UP SUMMARY TABLE -----
    # drop unecessary columns
    summary = summary.drop('BRNoise', axis=1)
    summary = summary.drop('BRConfidence', axis=1)
    summary = summary.drop('GSR', axis=1)
    summary = summary.drop('LinkQuality', axis=1)
    summary = summary.drop('RSSI', axis=1)
    summary = summary.drop('TxPower', axis=1)
    summary = summary.drop('ECGNoise', axis=1)
    summary = summary.drop('HRV', axis=1)

    # fill 0's in HR column with the previous value
    summary['HR'] = summary['HR'].mask(summary['HR'] == 0).ffill(downcast='infer')

    # convert Time column into datetime object
    summary['Time'] = summary['Time'].apply(lambda x: datetime.strptime(x, '%d/%m/%Y %H:%M:%S.%f'))



    # ----- TRIMMING GLUCOSE AND SUMMARY TABLES -----
    # TOP TRIM
    # find time that patient woke up on Day 2 (first time recorded in summary table)
    summary_start_time = summary['Time'].iloc[0]

    # filter glucose table to be start 5 minutes after summary_start_time
    glucose = glucose[glucose['Time'] > summary_start_time + timedelta(minutes=5)]

    # get the start time from raw_glucose_top_trim
    glucose_start_time = glucose['Time'].iloc[0]

    # go back to summary table and cut it back 5 minutes from the start of glucose table
    summary = summary[summary['Time'] > glucose_start_time - timedelta(minutes=5)]

    # BOTTOM TRIM
    # get the last time for Day 2 in glucose table
    # glucose_end_time = glucose['Time'].iloc[-1]

    # get the last time stamp in summary table - this is the time they go to sleep
    summary_end_time = summary['Time'].iloc[-1]

    # now we want to filter glucose to end 5 minutes after summary_end_time
    glucose = glucose[glucose['Time'] < summary_end_time + timedelta(minutes=5)]

    

    # ----- CLEANING ECG DATA -----
    # take the millisecond string from the the first entry
    millisecond_str = ecg['Time'].iloc[0][-3:]

    # filter for only times with that milisecond string
    ecg = ecg.loc[ecg['Time'].str[-3:] == millisecond_str]

    # turn the Time coulmn into datetime objects
    ecg['Time'] = ecg['Time'].apply(lambda x: datetime.strptime(x, '%d/%m/%Y %H:%M:%S.%f'))

    # align raw_ecg with the summary times
    new_summmary_start_time = summary['Time'].iloc[0]
    ecg = ecg[ecg['Time'] >= new_summmary_start_time]


    
    # ----- CLEANING BREATHING DATA -----
    # filter for only times with that milisecond string
    breathing = breathing.loc[breathing['Time'].str[-3:] == millisecond_str]

    # turn the Time coulmn into datetime objects
    breathing['Time'] = breathing['Time'].apply(lambda x: datetime.strptime(x, '%d/%m/%Y %H:%M:%S.%f'))

    # align with raw_summary data
    ecg = ecg[ecg['Time'] >= new_summmary_start_time]


    qualities = pd.concat([summary.set_index('Time'),
                            ecg.set_index('Time'),
                            breathing.set_index('Time')], axis=1, join='inner')

    return [qualities, glucose]


In [21]:
files = ["/Users/chelseahuynh/CGM_ltuple/patient1day2/2014_10_02-10_56_44_Summary.csv",
        "patient1day2/2014_10_02-10_56_44_ECG.csv",
        "patient1day2/2014_10_02-10_56_44_Breathing.csv",
        "/Users/chelseahuynh/CGM_ltuple/patient1day2/glucose.csv"]
trimCGM(files, 2)

,HR,BR,SkinTemp,Posture,Activity,PeakAccel,BatteryVolts,BatteryLevel,BRAmplitude,ECGAmplitude,...,SagittalMin,SagittalPeak,DeviceTemp,StatusInfo,CoreTemp,AuxADC1,AuxADC2,AuxADC3,EcgWaveform,BreathingWaveform
Time,,,,,,,,,,,,,,,,,,,,,
2014-10-02 10:59:00.420,90,7.4,-3276.8,-12,0.02,0.06,4.158,93,6.0,0.0008,...,0.09,0.18,26.0,531,36.8,416,414,462,1552,7500985
2014-10-02 10:59:01.420,90,7.4,-3276.8,-13,0.05,0.12,4.158,93,6.0,0.0008,...,0.10,0.20,26.0,531,36.8,415,421,483,1535,7501839
2014-10-02 10:59:02.420,90,7.5,-3276.8,-13,0.04,0.10,4.158,93,7.0,0.0008,...,0.08,0.20,26.0,531,36.8,413,429,484,1597,7499544
2014-10-02 10:59:03.420,90,7.5,-3276.8,-11,0.04,0.07,4.158,93,8.0,0.0008,...,0.07,0.21,26.0,531,36.8,415,421,483,1602,7497676
2014-10-02 10:59:04.420,90,7.7,-3276.8,-12,0.07,0.12,4.158,93,10.0,0.0008,...,0.08,0.22,26.0,531,36.8,415,423,485,1580,7509417
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2014-10-02 21:40:59.420,73,21.1,-3276.8,-78,0.22,0.44,3.804,34,335.0,0.0000,...,0.15,0.60,33.2,528,35.4,416,420,484,297,7652168
2014-10-02 21:41:00.420,73,20.2,-3276.8,-80,0.11,0.27,3.804,34,310.0,0.0008,...,0.33,0.46,33.2,531,35.4,411,417,481,297,7608890
2014-10-02 21:41:01.420,73,20.2,-3276.8,-73,0.23,0.32,3.804,34,366.0,0.0008,...,0.10,0.53,33.2,531,35.4,414,420,482,297,8105132
